## ENSEMBLE LEARNING AND RANDOM FOREST


**1. If you have trained five different models on the exact same training data, and
they all achieve 95% precision, is there any chance that you can combine these
models to get better results? If so, how? If not, why?**

Si, es probable que podamos obtener mejores resultados combinando estos modelos utilizando un Voting Classifier. Aunque los modelos independientes tengan una alta precisión, pueden cometer errores diferentes en instancias distintas. Si lo errores de los modelos no están correlacionados entre sí (independientes), la votación permite que los errores individuales se compensen entre sí. 

**2. What is the difference between hard and soft voting classifiers?**

La diferencia radica en cómo el ensamble decide la clase final. Mientras que el Hard Voting cuenta los votos de cada modelo individuales y elige la clase que recibió la mayoría de votos, el Soft Voting calcula el promedio de las probabilidades estimadas para cada clase por todos los modelos individuales para luego elegir la clase con la probabilidad promedio más alta. Esto implica que, para usarlo, todos los clasificadores individuales deben ser capaces de estimar probabilidades. 

**3. Is it possible to speed up training of a bagging ensemble by distributing it across
multiple servers? What about pasting ensembles, boosting ensembles, random
forests, or stacking ensembles?**

La posibilidad de distribuir el entrenamiento depende de si el algoritmo es independiente o secuencial: 

- Bagging, Pasting y Random Forest: si es posible ya que cada predictor de estos modelos se entrena de forma independiente a los demás utilizando diferentes subconjuntos de datos.
- Boosting: no se puede paralelizar fácilmente (o solo de forma limitada) ya que funcionan de forma secuencial, cada predictor se entrena para corregir los errores cometidos por su predecesor, por loq ue no pueden entrenarse al mimsmo tiempo en servidores distintos.
- Stacking ensembles: su entrenamiento es mayormente secuencial entre capas. Aunque los modelos dentro de una misma capa si podrían entrenarse en paralelo, el proceso global de pasar de una capa a otra es jerárquico y dependiente (los modelos de la capa 2 necesitan las predicciones de los modelos de la capa 1).

**4. What is the benefit of out-of-bag evaluation?**

El benificio principal es que permite evaluar el desempeño de un modelo de bagging sin necesidad de reservar un conjunto de validación por separado. Esto se debe a que, matemáticamente, cada predictori utiliza solo de media el 63% de las instancias de entrenamiento, por lo que el restante son instancias “limpias” que pueden usarse como conjunto de validación, muy útil si la cantidad de datos es escasa.

**5. What makes Extra-Trees more random than regular Random Forests? How can
this extra randomness help? Are Extra-Trees slower or faster than regular Ran‐
dom Forests?**

Meintras que un Random Forest buscar el mejor umbral posible apra cada característica al dividir un nodo, los Extra-Trees eligen umbrales al azar para cada característica y seleccionann el mejor de esos umbrales aleatorios para realizar la división. Este método supondrá un aumentode las vias pero una reducción de la varianza, asi como un aumento en la velocidad de entrenamiento.

**6. If your AdaBoost ensemble underfits the training data, what hyperparameters
should you tweak and how?**

La aparición del underfitting implica que el modelo es demasiado simple para capturar la estructura de los datos por lo que habrá que aumentar la complejidas del modelo a través de: 

- Aumento del n_estimators → incrementar el número de predictores secuenciales para que el modelo tenga más oportunidades de corregir los errores acumulados.
- Reducir la regularización del estimador base → Por ejemplo si usamos un árbol de decisión, aumentar su complejidas ya sea aumentando su profundidad máxima o reduciendo las restricciones de los leaf nodes.
- Aumentar el learning_rate → Ayuda a que cada para de corrección tenga un impacto más fuerte en el resultado final.

**7. If your Gradient Boosting ensemble overfits the training set, should you increase
or decrease the learning rate?**

El learning_rate se encarga de escalar la contribución de cada árbol que se añade al ensamble. Cuando se reduce este valor, cada árbol tiene un impacto menor en el resultado final y es una de las formas más efectivas de regularizar un modelo de Gradiente Boosting y asi evitar el overfitting. 

In [1]:
import numpy as np
from sklearn.datasets import make_moons, fetch_openml
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, ShuffleSplit
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.base import clone
from scipy.stats import mode
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.svm import LinearSVC, SVC

**8. Load the MNIST data , and split it into a training set, a
validation set, and a test set (e.g., use 50,000 instances for training, 10,000 for val‐
idation, and 10,000 for testing).**

In [2]:
#1. Cargar los datos
mnist = fetch_openml("mnist_784", version=1, as_frame=False)

#2. Separación features
X, y = mnist["data"], mnist["target"]

#3. División de sets
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=10000, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=10000, random_state=42)

**Then train various classifiers, such as a Random Forest classifier, an Extra-Trees classifier, and an SVM.**

In [3]:
#4. Entrenamiento de los modelos
# Instancia los modelos
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)
ext_clf = ExtraTreesClassifier(n_estimators=100, random_state=42)
svm_clf = SVC(probability=True, random_state=42) 

# Entrenar los modelos
models = [rnd_clf, ext_clf, svm_clf]
for model in models:
    model.fit(X_train, y_train)

**Next, try to combine them into an ensemble that outperforms them all on the validation set, using a soft or hard voting classifier.**

In [4]:
#5. Creación del Voting Classifier
named_models = [
    ("random_forest", rnd_clf),
    ("extra_trees", ext_clf),
    ("svm", svm_clf),
]

voting_clf = VotingClassifier(named_models, voting="soft") # Por defecto es hard voting
voting_clf.fit(X_train, y_train)

#6. Evaluación de los modelos en el set de validación
models_finals = [rnd_clf, ext_clf, svm_clf, voting_clf]
print("--EVALUACIÓN DE LOS MODELOS SET VALIDACIÓN")
for model in models_finals:
    # Obtenemos el nombre de la clase para identificarlo en el print
    name = model.__class__.__name__
    if name == "VotingClassifier":
        name = "VotingClassifier (Ensemble)"
        
    score = model.score(X_val, y_val)
    print(f"{name:25} -> Accuracy: {score:.4f}")

--EVALUACIÓN DE LOS MODELOS SET VALIDACIÓN
RandomForestClassifier    -> Accuracy: 0.9692
ExtraTreesClassifier      -> Accuracy: 0.9715
SVC                       -> Accuracy: 0.9788
VotingClassifier (Ensemble) -> Accuracy: 0.9791


**Once you have found one, try it on the test set. How much better does it perform compared to the individual classifiers?**

In [5]:
print("EVALUACIÓN DE LOS MODELOS SET DE PRUEBA")
#7. Predicciones en el test de prueba
for clf in models_finals:
    name = clf.__class__.__name__
    if name == "VotingClassifier":
        name = "SOFT VOTING (Ensemble)"
    
    # Calculamos el score en el Test Set
    test_score = clf.score(X_test, y_test)
    print(f"{name:25} -> Accuracy: {test_score:.4f}")


EVALUACIÓN DE LOS MODELOS SET DE PRUEBA
RandomForestClassifier    -> Accuracy: 0.9645
ExtraTreesClassifier      -> Accuracy: 0.9691
SVC                       -> Accuracy: 0.9760
SOFT VOTING (Ensemble)    -> Accuracy: 0.9767


**9. Run the individual classifiers from the previous exercise to make predictions on the validation set, and create a new training set with the resulting predictions: each training instance is a vector containing the set of predictions from all your classifiers for an image, and the target is the image’s class. Train a classifier on this new training set. Congratulations, you have just trained a blender, and together with the classifiers they form a stacking ensemble! Now let’s evaluate the ensemble on the test set. For each image in the test set, make predictions with all your classifiers, then feed the predictions to the blender to get the ensemble’s predictions. How does it compare to the voting classifier you trained earlier?**

In [6]:
#1.  Creamos una matriz vacía para las nuevas características
X_val_predictions = np.empty((len(X_val), len(models)), dtype=np.float32)

#2. Llenamos la matriz con las predicciones de cada modelo
for index, model in enumerate(models):
    X_val_predictions[:, index] = model.predict(X_val)

#3. Instancia y entrenamiento del modelo
rnd_forest_blender = RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42)
rnd_forest_blender.fit(X_val_predictions, y_val)

#4. Generar predicciones de la capa base sobre X_test
X_test_predictions = np.empty((len(X_test), len(models)), dtype=np.float32)

for index, model in enumerate(models):
    X_test_predictions[:, index] = model.predict(X_test)

#5. Predicción final
y_pred_final = rnd_forest_blender.predict(X_test_predictions)

#6. Evaluación de la precisión
print(f"Precisión del Stacking (Blender): {accuracy_score(y_test, y_pred_final):.4f}")

Precisión del Stacking (Blender): 0.9697
